# Read me!!

Hello, welcome to my notebook!!

This notebook is for you who want to know how you can load and handle dataset.

The key points are as follows:

1. You have to reduce the memory to allocate your dataset.

> There are so many dataset which have large size.

> So, you may convert each data in row into more small size (byte).

2. EDA will be conducted briefly.

> There are plenty of columns in dataset, but we just identify what column has NaN value or outlier (it will be enough!!).

3. We will divide the columns into dummy cluster and non-dummy cluster.

> This dataset includes some dummy variables and not.

> So, we will divide the dataset into two clusters for our future model (see [Parallel startegy through deep learning])!!


If you have any questions, please leave the comments.

I hope you to gain more imformation about data handling, DNN, CNN, and etc..

## **Knowledge can be improved by being shared.**

Please upvote!!


## [You can learn more skills for handling dataset or neural network.]

### [Parallel strategy through deep learning (0.956%)] - Tabular Playground Series - Dec 2021
- https://www.kaggle.com/pythonash/parallel-strategy-through-deep-learning-0-956

### [How to use csv and img at the same time] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/how-to-use-csv-and-img-at-the-same-time

### [Parallel combination DNN with CNN] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/parallel-dnn-and-cnn-network-for-beginners
 
### [Image data handling without memory exploded] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/how-to-handle-dataset-for-beginners

### [Data handling & Deep learning] - Titanic competition (Top 7%)
 - https://www.kaggle.com/pythonash/how-to-handle-raw-dataset-and-analyze-with-dl
 
### [Deep learning model with SeLU activation function] - Titanic competition
- https://www.kaggle.com/pythonash/selu-activation-function-in-dl

### [Preparing a completed dataset with proper imputation method] - Titanic competition
 - https://www.kaggle.com/pythonash/making-completed-dataset

**Let's start!**

# Data loading and handling.

## Import some useful libraries.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold as SK
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score



train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')

test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')

## Making your data more light with smaller size.

In [ ]:
def reduce_mean_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] =='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min> np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print('Memory usage is decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem)/ start_mem))
    
    return df

In [ ]:
train = reduce_mean_usage(train)
test = reduce_mean_usage(test)

## Sampling your data with randomness.

It is crucial part for using deep learning or even predicting because the randomness will make your model be more robust and have more generalization power.

In [ ]:
# Use sample method with frac =1 .
train = train.sample(frac=1).reset_index(drop=True)

train_x = train.drop(['Id','Cover_Type'],axis=1)

test_x = test.drop(['Id'],axis=1)

train_y = train['Cover_Type']

train

## Identifying teh dataset whether there is NaN value or not.

In [ ]:
train_x.isnull().sum()

## We don't need to imputate any values!!

In [ ]:
train_y.isnull().sum()

## Identifying the target variable's distribution.

There are total 7 classes.

However, the class 5 is only one in target variable.

So, you should delete the class because it can be regarded as outlier when the model conducts predicting.

> Your model will compute the weights or train the values with the class 5.

In [ ]:
train_y.value_counts()

## Drop the class 5!!

In [ ]:
drop_5_index = train_y[train_y == 5].index[0]

train_x = train_x.drop([drop_5_index],axis=0).reset_index().drop(['index'],axis=1)
train_y = train_y.drop([drop_5_index],axis=0).reset_index().drop(['index'],axis=1)

## Descriptive statistics for train input.

As you can see, the variables consist of dummy variable and non-dummy variable.

So, we will divide the dataset into dummy cluster and non-dummy cluster.

In [ ]:
train_x.describe()

## Processed target variable distribution.

In [ ]:
train_y.value_counts()

## For using deep learning, scale your data with min-max scale.

In [ ]:
scaler = MinMaxScaler()
train_df = scaler.fit_transform(train_x)
train_df = pd.DataFrame(train_df)
train_df.columns = train_x.columns
train_df

In [ ]:
test_df = pd.DataFrame(scaler.transform(test_x))
test_df.columns = test_x.columns
test_df

In [ ]:
train_df.columns

## Deviding the dataset into dummy and non-dummy cluster.

In [ ]:
train_non_dummy, train_dummy = train_df.columns[:10], train_df.columns[10:]

train_df_dummy = train_df[train_dummy]
train_df_non_dummy = train_df[train_non_dummy]

In [ ]:
test_df_dummy = test_df[train_dummy]
test_df_non_dummy = test_df[train_non_dummy]

## Making the previous data empty.

> For using memory more free, make the of no use data empty.

In [ ]:
train=[]
test=[]
train_x=[]
test_x=[]
train_df=[]
test_df=[]

## Lastly, encoding the target variable with 6 classes.

In this case, when you use deep learing with keras, you have to use 'sparse_categorical_crossentropy' and 'sparse_categorical_accuracy' as loss function and metric indicator, repectively.

In [ ]:
Encoder = LabelEncoder()

y_encoded = Encoder.fit_transform(train_y)
train_y = []
y_encoded

# It has done!!

Until now, you have learned about loading and handling the large data.

The next step is prepared for predicting and submitting the result.

And I will introduce my notebook with 27% rating through deep learing model.

The reason why we divide dataset into dummy and non-dummy cluster will be described in my next notebook (I will call this as parallel startegy.

If you get any helps from my notebook, please upvote!!

Thank you!